In [1]:
import cv2
import numpy as np

In [2]:
net=cv2.dnn.readNet(r"C:\Users\Asus\yolov3.weights",r"C:\Users\Asus\yolov3.cfg")

In [3]:
classes=[]

In [4]:
with open('coco.names','r') as f:
    classes=f.read().splitlines()

In [5]:
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [34]:
img=cv2.imread("D:\Object_Count_Test_data\Train\workingimage.jpg")
raw_height,raw_width, _ =img.shape



In [35]:
print(img.shape)

(900, 1575, 3)


In [36]:
blob=cv2.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True,crop=False)

In [37]:
net.setInput(blob)
output_layers_names=net.getUnconnectedOutLayersNames()
finaloutputs=net.forward(output_layers_names)

In [38]:
boxes_no=[]
confidences_score=[]
class_ids=[]
for outputs in finaloutputs:
    for detect in outputs:
        scores=detect[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence > 0.7:
            x_center=int(detect[0]*raw_width)
            y_center=int(detect[1]*raw_height)
            w=int(detect[2]*raw_width)
            h=int(detect[3]*raw_height)
            
            x=int(x_center-w/2)
            y=int(y_center-h/2)
            boxes_no.append([x,y,w,h])
            confidences_score.append((float(confidence)))
            class_ids.append(class_id)
        
   # print(len(boxes_no))   
    index=cv2.dnn.NMSBoxes(boxes_no,confidences_score,0.7,0.4)
   # print(index.flatten())

In [39]:
def countobject(whole_track_list,index_of_track):
    count=0
    for p in whole_track_list:
        
            
            if(p==index_of_track):
                
                count+=1
    return count

In [40]:
def numOfsameIndex(whole_redundant_list,item_id):
    count1=0
    for q in whole_redundant_list:
        if(q==item_id):
            count1+=1
    return count1
           
        


In [41]:
colors=np.random.uniform(0,255,size=(len(boxes_no),3))

In [42]:
track=[]
for i in index.flatten():
    
    x,y,w,h=boxes_no[i]
    label=str(classes[class_ids[i]])
    #print(label)
    #print(class_ids[i])
    track.append(class_ids[i])
   
    confidence=str(round(confidences_score[i],2))
    color=colors[i]
    
    cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
    
    cv2.putText(img,label,(x,y+20),cv2.FONT_HERSHEY_SIMPLEX,1,(100,255,90),1)
   

In [43]:
print(track)

[56, 63, 56, 56, 56, 0, 0, 0]


In [44]:
redundant=[]
countlists=[]
size=len(track)

#print(size)
#print(redundantsize)

i=0
while i<size:
    
    redundantsize=len(redundant)
    #print("lenth of redundant array",redundantsize)
    if (redundantsize==0):
        totalobject=countobject(track,track[i])
        #print("object index and total number of occurace is",track[i], totalobject)
        redundant.append(track[i])
        countlists.append(totalobject)
        i+=1
        
    else:
        rslt=numOfsameIndex(redundant,track[i])
        
        if(rslt==0):
            totalobject=countobject(track,track[i])
            #print("object id and total number of occurace is",track[i], totalobject)
            #text="obj1".format(totalobject)
            #cv2.putText(img,text,(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1)
            redundant.append(track[i])
            countlists.append(totalobject)
            i+=1
        else:
            i+=1
            
        
            
    
    
        



In [45]:
loopsize=len(countlists)

In [46]:

i=0
x=20
y=30
while i<loopsize:
    label=str(classes[redundant[i]])
    cv2.putText(img,label,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1)
    x+=120
    text=":{}".format(countlists[i])
    cv2.putText(img,text,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1)
    i+=1
    x=x-120
    y+=30
    
    

In [47]:


cv2.imshow('Image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()